In [1]:
import numpy as np
import pandas as pd
import glob
import sys,os

In [2]:
from pathlib import Path

### API

In [3]:
sys.path.append('../../')

In [4]:
from utils.bbox.conversation import bbox_coco2voc, bbox_voc2yolo

In [5]:
def bbox_voc2yolo_df(df):
    w =  df['width'].values
    h =  df['height'].values
    w_bbox = df['bx3'].values - df['bx1'].values
    h_bbox = df['bx4'].values - df['bx2'].values
    x_c = df['bx1'].values + w_bbox/2
    y_c = df['bx2'].values + h_bbox/2
    df.loc[:, 'bx1'] = x_c/w
    df.loc[:, 'bx2'] = y_c/h
    df.loc[:, 'bx3'] = w_bbox/w
    df.loc[:, 'bx4'] = h_bbox/h    
    return df

### Dataset Information

In [11]:
root_nas = Path('./data/hwpedia')

In [13]:
!ls $root_nas

hwpedia70k_slink_df.csv  hwpedia_bbox.csv  hwpedia_slink_df.csv


In [17]:
bboxes_df = pd.read_csv(root_nas /'hwpedia_bbox.csv')
# hwpedia_del = pd.read_csv('./data/exps/dadetGallv4_del.csv')
# hwpedia_cat = pd.read_csv('./data/hwpedia_bbox.csv')
# bboxes_df = bboxes_df[bboxes_df['label_pred']>=0]
# hwpedia_cat.to_csv('./data/hwpedia_bbox.csv', index=False)

/tmp/ipykernel_112310/681476183.py:1: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  bboxes_df = pd.read_csv(root_nas /'hwpedia_bbox.csv')


In [2]:
# bboxes_df.head(1)

In [20]:
bboxes_df['subffix'] = bboxes_df['img_sp'].apply(lambda x: Path(x).suffix)

In [21]:
bboxes_df['subffix'].value_counts()

.jpg    269575
Name: subffix, dtype: int64

In [9]:
from tqdm import tqdm
from PIL import Image
w_list = []
h_list = []
for p in tqdm(bboxes_df['img_sp'].values):
    img = Image.open(p)
    w,h = img.size
    w_list.append(w)
    h_list.append(h)
    # break

100%|█████████████████████████████████| 312242/312242 [00:26<00:00, 11822.65it/s]


In [10]:
bboxes_df['width'] = w_list
bboxes_df['height'] = h_list

In [11]:
# bboxes_df.to_csv(root_nas /'bboxes/dadet100k/dadet100kv5_bboxes.csv', index=False)

In [ ]:
label2model = get_list2dict( model_label_map.values(), model_label_map.keys())

In [16]:
hwpedia_cat['stem'] = hwpedia_cat['img_sp'].apply(lambda x: Path(x).stem)

In [24]:
df_val = hwpedia_cat[hwpedia_cat['mode']=='val']
len(df_val)

32183

In [25]:
df_train = hwpedia_cat[hwpedia_cat['mode']=='train']
len(df_train)

187392

In [27]:
187392/(187392+50000)

0.7893779065848892

In [20]:
df_train_sample = df_train.sample(int(len(df_train)-5e4))

In [21]:
df = pd.concat([df_train_sample, df_val], axis=0)

In [22]:
# df.to_csv('./data/exps/dadetGallv4_del.csv', index=False)